# Imports

In [ ]:
import numpy as np

# Create Compatability Matrix

In [24]:
def generate_random_compatibility_matrix(num_clusters_method_a, num_clusters_method_b, constant_sum):
    random_matrix = np.zeros((num_clusters_method_a, num_clusters_method_b), dtype=int)
    cluster_a_list = np.zeros((num_clusters_method_a), dtype=int)

    for row_index in range(constant_sum):
        random_index = np.random.randint(0, num_clusters_method_a)
        cluster_a_list[random_index] += 1
    print("Cluster A sizes: ", cluster_a_list)

    for row_index in range(num_clusters_method_a):
        for _ in range(cluster_a_list[row_index]):
            random_index = np.random.randint(0, num_clusters_method_b)
            random_matrix[row_index, random_index] += 1

    return random_matrix


num_clusters_method_a = 4
num_clusters_method_b = 4
total_members = 100

compatibility_matrix = generate_random_compatibility_matrix(num_clusters_method_a, num_clusters_method_b, total_members)

print("Compatibility Matrix:\n", compatibility_matrix)


Cluster A sizes:  [29 28 25 18]
Compatibility Matrix:
 [[ 4 10  7  8]
 [ 3  5 12  8]
 [ 8  5  7  5]
 [ 4  3  3  8]]
